<a href="https://colab.research.google.com/github/nreichen/eecs448-CollegeSubredditsCommentGeneration/blob/main/subredditValidation/umd_test_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models/UMD"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['checkpoints', 'train_post_set.pickle', 'val_post_set.pickle', 'test_post_set.pickle', 'train_com_set.pickle', 'val_com_set.pickle', 'test_com_set.pickle', 'UMD_model.ipynb', 'umd_test_val.ipynb']


In [ ]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/UMD


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/UMD


In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers
!pip install convokit
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.4 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=

In [ ]:
import pickle
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import nltk
# maybe modify the above to a different model? idk up to you all
# also how to use the gpu command? like the cuda stuff.
# 
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
import torch
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load and Prepare Dataset


In [ ]:
seed = 448
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/UMD


In [ ]:
sub_reddit = '/Cornell'

In [ ]:
val_post_path = os.path.join(GOOGLE_DRIVE_PATH, "val_post_set.pickle")
val_com_path = os.path.join(GOOGLE_DRIVE_PATH, "val_com_set.pickle")

In [ ]:
print(val_post_path)

drive/My Drive/448 Group: Jedi Order/models/UMD/val_post_set.pickle


In [ ]:
# posts_datasets_to_use[0] = pickle.load(open(train_post_path, "rb"))
val_posts = pickle.load(open(val_post_path, "rb"))
# posts_datasets_to_use[2] = pickle.load(open(test_post_path, "rb"))
# comments_datasets_to_use[0] = pickle.load(open(train_com_path, "rb"))
val_comments = pickle.load(open(val_com_path, "rb"))
# comments_datasets_to_use[2] = pickle.load(open(test_com_path, "rb"))

In [ ]:
for pt in val_posts:
  print(pt)

<bos> I'm an incoming freshman and I want to start lifting and I know nothing about it. How easy will that be at UMD resource wise(gyms, access to personal trainers possibly, the right food, etc)? Did any of you start lifting when you came here? How did you go about it? <eos> 
<bos> Hey everyone just want some quick opinions on my schedule MATH461 Linear Algebra (Grillakis) MATH241 Calc 3 (TBA maybe Justin) CMSC132 Object Oriented programming 2 (Nelson) CMSC250 Discrete Math (Jason) STAT430 SAS programming (Christine Nickel) By no means is this the toughest schedule out there but it is 18 credits and I was wondering if I should just push one of these classes for another semester or in the summer cuz I want to have some social life. The teachers I have are all pretty good and the classes individually are not too bad. But I just want some opinions or if anyone has experience with a similar schedule. Also any info about any of the instructors would be much appreciated to.  <eos> 
<bos> I'

In [ ]:
print(val_posts)

["<bos> I'm an incoming freshman and I want to start lifting and I know nothing about it. How easy will that be at UMD resource wise(gyms, access to personal trainers possibly, the right food, etc)? Did any of you start lifting when you came here? How did you go about it? <eos> ", '<bos> Hey everyone just want some quick opinions on my schedule MATH461 Linear Algebra (Grillakis) MATH241 Calc 3 (TBA maybe Justin) CMSC132 Object Oriented programming 2 (Nelson) CMSC250 Discrete Math (Jason) STAT430 SAS programming (Christine Nickel) By no means is this the toughest schedule out there but it is 18 credits and I was wondering if I should just push one of these classes for another semester or in the summer cuz I want to have some social life. The teachers I have are all pretty good and the classes individually are not too bad. But I just want some opinions or if anyone has experience with a similar schedule. Also any info about any of the instructors would be much appreciated to.  <eos> ', "

Load the trained model

In [ ]:
model_file_path = "checkpoints"

In [ ]:
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, model_file_path)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

^^^^^^run if needed 

In [ ]:
#https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-
# post_in = "I want to go to hotel management school for rural bed and breakfasts. What experiences or classes should I seek out?"
post_in = "Hey everyone! For those of you who took BIOMI 2900, or General Microbiology, did you take the discussion? If so, did it help at all? Trying to decide if I should or not. I'm not a microbiology major but I'm in a microbio lab at the moment so some of it does make sense already. Thanks!"
post_with_bos = post_in + " TL;DR"
encoded_prompt = tokenizer(post_in, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

# prediction
output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=3
)

generated_sequences = []
print(len(output_sequences))
torch.unique(output_sequences, dim=0)
print(len(output_sequences))

# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    #if not ALLOW_NEW_LINES:
        #limit = text.find('\n')
        #text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3
3


In [ ]:
# print(generated_sequences)
for sent in generated_sequences:
  print(sent.replace(post_in, '').replace('TL;DR', ''))

   The professor told me that the discussion was more about what he did than anything about the class.    Also, for the course, was there any sort of pressure to do a lot of reading? You'll learn a lot from discussions with your fellow students. For microbiology, you're going to learn a lot of stuff that
   You could do something similar for Gen Microbiology right now. I'm in the Bio and Bio-Physics department so my experience would probably vary depending on who's taking it, but I've heard pretty good things about the lab. I guess it's just a very different type of lab, though: it's open to anybody interested. I
   If you had an interesting experience with the class, did you take any chemistry class? I took it during the last school year. It took me a few days to find out everything about the theory, and a little while back I looked into the class. I did not get a bad grade on the midterm exam, but I was fairly surprised.


In [ ]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=52fedd09dde98b8a4cfd447564d8e101003ff9da12592e47e80c1802c584cab2
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
from torch import optim
from torch.utils.data import Dataset
from tqdm import tqdm

In [ ]:
# print(post_tensors_data[1])
# print(comments_datasets_to_use[1])
def validation_generate(post):
  # print(post)
  # post_with_bos = "<bos>" + post + " TL;DR"
  post_with_bos = post + " TL;DR"
  encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
  encoded_prompt = encoded_prompt.to(model.device)
  # print(encoded_prompt)

  # prediction
  output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=1
  )
    
  generated_sequences = []
  # decode prediction
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    generated_sequences.append(text.strip())
  
  # print(generated_sequences)
  return generated_sequences

In [ ]:
val_len = len(val_comments)
val_dataset = [{} for sub in range(val_len)]

for cnt, item in enumerate(val_comments):
  val_dataset[cnt]['reference_summary'] = item.replace("<bos>", "").replace("<eos>", "")


print(val_dataset[1]['reference_summary'])

 ur gonna die I’m sorry mate Looooooook some idiot is going down the eviscerator. 18 credits is difficult, especially considering the classes you’re taking. I would drop stat or one of the math classes. If Justin is teaching 241, I would keep it. Yeah I have to say that this is an extremely tough schedule just because of the completely new material that you’ll be learning in linear and 132. Not to say that those classes are impossible they take together, but putting them with calc 3 and stat is kinda asking for someone to make your asshole so exposed that anyone can take advantage of it at a moment’s notice. Linear actually requires reading the book to figure out the logic and very little computation compared to calc and 132 just takes time due to the nature of comp sci work. I’d suggest dropping one or two courses.  


In [ ]:
for cnt, post in enumerate(val_posts):
  # print(post)
  generated_sequences = validation_generate(post)

  for sent in generated_sequences:
    sent = sent.replace(post, '').replace('TL;DR', '')
    val_dataset[cnt]['generated_text'] = sent
    # print(sent)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

**Calculating the Rouge score**

In [ ]:
!pip3 install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
generated_text = val_dataset[7]['generated_text']
reference_summary = val_dataset[7]['reference_summary']
print("GENERATED COMMENTS: {}\nReference COMMENTS:{}".format(generated_text, reference_summary))

GENERATED COMMENTS: I'm coming to UMD in the fall, and I have a dumb question. I go to the gym a lot, and after the gym i have a protein shake. At UMD, how would I make my protein shakes and then clean the bottle/container... Would i bring a ninja blender?? or a blender bottle? and where would i clean my dishes and shit    I would bring a ninja blender. I think it's the best option.  If you put you in the laundry room you can actually do that, and if you clean out that the dishes are on your table. I took a ton of practice exams with this one.  If I am honest, if I clean up my dishwasher there is no way it can properly
Reference COMMENTS: You clean your shaker cup? You're wasting so much protein. boi All the dorms afaik have a sink in the lounge where you can do dishes. Let's say you have a ninja, bring a sponge and dish soap. Then was the blade and bottle in the bathroom sink.  I'll be looking for a gym bro next semester. Hit me up. If your dorm lacks a kitchen area, you can always ju

In [ ]:
from rouge import Rouge

# create a Rouge object
rouge = Rouge()

# initialize scores for each variant of ROUGE
scores_rouge1 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rouge2 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rougeL = {'recall': 0, 'precision': 0, 'fmeasure': 0}

for idx in range(val_len):
    generated_text = val_dataset[idx]['generated_text']
    reference_summary = val_dataset[idx]['reference_summary']
    
    scores = rouge.get_scores(generated_text, reference_summary, avg=True)
    
    # compute ROUGE-1 scores
    scores_rouge1['recall'] += scores['rouge-1']['r']
    scores_rouge1['precision'] += scores['rouge-1']['p']
    scores_rouge1['fmeasure'] += scores['rouge-1']['f']
    
    # compute ROUGE-2 scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_n=(2, 2))
    scores_rouge2['recall'] += scores['rouge-2']['r']
    scores_rouge2['precision'] += scores['rouge-2']['p']
    scores_rouge2['fmeasure'] += scores['rouge-2']['f']
    
    # compute ROUGE-L scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_l=True)
    scores_rougeL['recall'] += scores['rouge-l']['r']
    scores_rougeL['precision'] += scores['rouge-l']['p']
    scores_rougeL['fmeasure'] += scores['rouge-l']['f']

# compute the average scores over all examples
n_examples = val_len
scores_rouge1 = {k: v/n_examples for k, v in scores_rouge1.items()}
scores_rouge2 = {k: v/n_examples for k, v in scores_rouge2.items()}
scores_rougeL = {k: v/n_examples for k, v in scores_rougeL.items()}

# print the overall scores
print("Overall ROUGE-1 scores:", scores_rouge1)
print("Overall ROUGE-2 scores:", scores_rouge2)
print("Overall ROUGE-L scores:", scores_rougeL)

Overall ROUGE-1 scores: {'recall': 0.24246514650249437, 'precision': 0.1785361011570437, 'fmeasure': 0.1906399860733472}
Overall ROUGE-2 scores: {'recall': 0.029253145759050896, 'precision': 0.02062117578898065, 'fmeasure': 0.021825668003748223}
Overall ROUGE-L scores: {'recall': 0.2148553311964727, 'precision': 0.15772711486510219, 'fmeasure': 0.16843159323727983}


**Calculating the sentiment analysis accuracy**

In [ ]:
!pip3 install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 14.1 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def find_label(scores):
  if scores['pos'] > scores['neg'] and scores['pos'] > scores['neu']:
    dominant_label = 'positive'
  elif scores['neg'] > scores['pos'] and scores['neg'] > scores['neu']:
    dominant_label = 'negative'
  else:
    dominant_label = 'neutral'
  
  return dominant_label

In [ ]:
acc = 0
sid = SentimentIntensityAnalyzer()
for idx in range(val_len):
  # print(val_dataset[idx]['reference_summary'])
  # print(type(val_dataset[idx]['reference_summary']))
  sent_gt = sid.polarity_scores(val_dataset[idx]['reference_summary'])
  sent_generated_comment = sid.polarity_scores(val_dataset[idx]['generated_text'])

  gt_label = find_label(sent_gt)
  generated_comment_label = find_label(sent_generated_comment)

  if gt_label == generated_comment_label:
    acc += 1

  # print("Reference comments: {}".format(val_dataset[idx]['reference_summary']))
  # print(sent_gt)
  # print(gt_label)
  # print("Generated comments: {}".format(val_dataset[idx]['generated_text']))
  # print(sent_generated_comment)
  # print(generated_comment_label)
  # if idx == 15:
  #   break

print("The number of generated comments that have aligned sentiments: {}".format(acc))
print("The number of posts in the validation set: {}".format(val_len))
print("The accuracy of sentiments of the generated comments: {}".format(acc / val_len))

The number of generated comments that have aligned sentiments: 2373
The number of posts in the validation set: 2392
The accuracy of sentiments of the generated comments: 0.992056856187291


**Readability validation test**

Automated readability index

In [ ]:
!pip3 install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.0 MB/s eta 0:00:00


Flesch-Kincaid Grade Level: The Flesch-Kincaid Grade Level measures the approximate grade level of a text, based on its sentence length and average number of syllables per word.

In [ ]:
import textstat

In [ ]:
grade_level = 0.0
for idx in range(val_len):
  sc = textstat.flesch_kincaid_grade(val_dataset[idx]['generated_text'])
  # print(sc)
  grade_level += sc

grade_level /= val_len
print("Flesch-Kincaid Grade Level:", grade_level)

Flesch-Kincaid Grade Level: 5.650794314381282


Gunning Fog Index: The Gunning Fog Index measures the approximate years of education needed to understand a text, based on its sentence length and percentage of complex words.

In [ ]:
fog_index = 0.0
for idx in range(val_len):
  sc = textstat.gunning_fog(val_dataset[idx]['generated_text'])
  # print(sc)
  fog_index += sc

fog_index /= val_len
print("Gunning Fog Index:", fog_index)

Gunning Fog Index: 7.79706103678929


Automated Readability Index (ARI): The Automated Readability Index measures the approximate grade level of a text, based on its average number of characters per word and average number of words per sentence.



In [ ]:
ari = 0.0
for idx in range(val_len):
  sc = textstat.automated_readability_index(val_dataset[idx]['generated_text'])
  # print(sc)
  ari += sc

ari /= val_len
print("Automated Readability Index:", ari)

Automated Readability Index: 6.178553511705678
